In [33]:
import pickle

import pandas as pd
from nltk import RegexpTokenizer
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline



In [34]:
#Data Loading

phish_data = pd.read_csv('dataset/phishing_site_urls.csv')

In [35]:
phish_data.head()

,URL,Label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,bad
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,bad
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,bad
3,mail.printakid.com/www.online.americanexpress....,bad
4,thewhiskeydregs.com/wp-content/themes/widescre...,bad


In [36]:
phish_data.tail()


,URL,Label
549341,23.227.196.215/,bad
549342,apple-checker.org/,bad
549343,apple-iclods.org/,bad
549344,apple-uptoday.org/,bad
549345,apple-search.info,bad


In [37]:
phish_data.isnull()

,URL,Label
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
549341,False,False
549342,False,False
549343,False,False
549344,False,False


In [38]:
phish_data.size

1098692

In [39]:
#Lets use Tokenizer first to gather words
# then use countVectorizer to Vectorize all words

tokenizer = RegexpTokenizer(r'[A-Za-z]+')
phish_data['text_tokenized'] = phish_data.URL.map(lambda t: tokenizer.tokenize(t))

KeyboardInterrupt: 

In [ ]:
# Snow Ball Streamer for getting root words

stemmer = SnowballStemmer(language='english')
phish_data['text_stemmed'] = phish_data['text_tokenized'].map(lambda l:
                                                              [stemmer.stem(word) for word in l])


In [ ]:
# Joining the stemmed text corpus

phish_data['text_sent'] = phish_data['text_stemmed'].map(lambda l: ' '.join(l))

In [ ]:
# Using Count Vectorization to vectorize the text corpus

cv = CountVectorizer()
feature = cv.fit_transform(phish_data['text_sent'])


In [ ]:
# Spliting training data and testing data

trainX, testX, trainY, testY = train_test_split(feature, phish_data.Label)

In [ ]:
# Using Logistic Regression to train the model

lr = LogisticRegression()
lr.fit(trainX, trainY)
print(lr.score(testX, testY))

In [ ]:
# Used to Predict the user input

def user_data_predict(user_data):
    res = lr.predict(user_data)[0]
    return res

In [ ]:
# Takes User Input and applies data pre-processing
# tasks before predicting

def user_input(data):
    tk_data = tokenizer.tokenize(data)
    stm_data = None
    join_data = []
    for word in tk_data:
        stm_data = stemmer.stem(word)
        join_data.append(stm_data)
    join_data = ' '.join(join_data)
    print(join_data)
    user_data = cv.transform([join_data])
    res = user_data_predict(user_data)
    return res

In [40]:
# Pipelining the code

pipeline_lr = make_pipeline(CountVectorizer(tokenizer = RegexpTokenizer(r'[A-Za-z]+').tokenize,stop_words='english'), LogisticRegression())
trainX, testX, trainY, testY = train_test_split(phish_data.URL, phish_data.Label)
pipeline_lr.fit(trainX,trainY)
print(pipeline_lr.score(testX,testY) )

C:\Users\N.K.Rahul\.virtualenvs\PhishMLmodel-7CoL9bgq\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\N.K.Rahul\.virtualenvs\PhishMLmodel-7CoL9bgq\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9668770979415598


In [41]:
# pickling the model
pickle.dump(pipeline_lr,open('model.pkl','wb'))